Steady state heat equation
======

This notebook will setup and solve the steady state heat equation:

$$
\nabla(k\nabla u) = f
$$

where $k$ is the diffusivity, T the temperature field and $h$ the source term.

**Keywords:** initial conditions, boundary conditions, heat equation

In [1]:
#|  echo: false  # Hide in html version

# This is required to fix pyvista 
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio
nest_asyncio.apply()

In [2]:
#| output: false # Suppress warnings in html version

import underworld3 as uw
import numpy as np
import sympy

[MacBook-Pro.local:31665] shmem: mmap: an error occurred while determining whether or not /var/folders/0j/bnxlsh897sl6b1rv06fnt5r80000gp/T//ompi.MacBook-Pro.502/jf.0/2453798912/sm_segment.MacBook-Pro.502.92420000.0 could be created.


#### Setup scaling of model with units

In [3]:
u = uw.scaling.units

### make scaling easier
ndim = uw.scaling.non_dimensionalise
nd   = uw.scaling.non_dimensionalise
dim  = uw.scaling.dimensionalise 


model_length = 20. * u.centimeter
model_height = 10. * u.centimeter
top_Temp = 273.15 * u.degK
base_Temp = 1603.15 * u.degK

KL = model_height
KT = (base_Temp - top_Temp)



scaling_coefficients  = uw.scaling.get_coefficients()
scaling_coefficients["[length]"] = KL
scaling_coefficients["[temperature]"]= KT

scaling_coefficients

[mass],1.0 kilogram
[length],0.1 meter
[temperature],1330.0 kelvin
[time],31557600.0 second
[substance],1.0 mole


#### Setup the mesh

In [4]:
# mesh = uw.meshing.UnstructuredSimplexBox(
#             minCoords=(nd(0. * u.centimeter), nd(0. * u.centimeter)), 
#             maxCoords=(nd(20. * u.centimeter), nd(10. * u.centimeter)),
#             cellSize=1 / 12
# )

mesh = uw.meshing.StructuredQuadBox(
            minCoords=(nd(0. * u.centimeter), nd(0. * u.centimeter)), 
            maxCoords=(nd(20. * u.centimeter), nd(10. * u.centimeter)),
            elementRes=(32,32)
)

x, y = mesh.X

mesh.view()

Structured box element resolution 32 32


Mesh # 0: .meshes/uw_structuredQuadBox_minC(0.0, 0.0)_maxC(2.0, 1.0).msh

No variables are defined on the mesh

| Boundary Name            | ID    | Min Size | Max Size |
| ------------------------------------------------------ |
| Bottom                   | 11    | 63       | 63       |
| Top                      | 12    | 63       | 63       |
| Right                    | 13    | 63       | 63       |
| Left                     | 14    | 63       | 63       |
| Null_Boundary            | 666   | 1089     | 1089     |
| All_Boundaries           | 1001  | 128      | 128      |
| All_Boundaries           | 1001  | 128      | 128      |
| UW_Boundaries            | --    | 1469     | 1469     |
| ------------------------------------------------------ |


DM Object: uw_.meshes/uw_structuredQuadBox_minC(0.0, 0.0)_maxC(2.0, 1.0).msh 1 MPI process
  type: plex
uw_.meshes/uw_structuredQuadBox_minC(0.0, 0.0)_maxC(2.0, 1.0).msh in 2 dimensions:
  Number

#### Visualise the mesh

In [5]:
from mpi4py import MPI

if MPI.COMM_WORLD.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)

    pl = pv.Plotter(window_size=(1000, 500), shape=(1, 1))


    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        use_transparency=False,
        opacity=1,
        show_scalar_bar=True,
    )

    pl.show()


Widget(value='<iframe src="http://localhost:61510/index.html?ui=P_0x318ee0100_0&reconnect=auto" class="pyvista…

In [6]:
# mesh variables
T_soln = uw.discretisation.MeshVariable(varname="T", mesh=mesh, num_components=1)


### The Poisson Solver

There are a number of pre-defined *solver systems* defined in `underworld3` 
which are templates for orchestrating the underlying PETSc objects. 
A solver requires us to define the unknown in the form of `meshVariables`, 
provide boundary conditions, a constitutive model, 
and provide `uw.functions` to define constitutive
properties, and driving terms.

We will use the `Poisson` solver for the diffusion equation, and we will 
use a `Projection` solver to compute the vertical gradient term. 

The solver classes themselves are documented, so we can figure out what 
is needed before we define anything:


In [7]:
uw.systems.Poisson.view()


This class provides functionality for a discrete representation
of the Poisson equation

$$
\nabla \cdot
        \color{Blue}{\underbrace{\Bigl[ \boldsymbol\kappa \nabla u \Bigr]}_{\mathbf{F}}} =
        \color{Maroon}{\underbrace{\Bigl[ f \Bigl] }_{\mathbf{f}}}
$$

The term $\mathbf{F}$ relates the flux to gradients in the unknown $u$

## Properties

  - The unknown is $u$

  - The diffusivity tensor, $\kappa$ is provided by setting the `constitutive_model` property to
one of the scalar `uw.constitutive_models` classes and populating the parameters.
It is usually a constant or a function of position / time and may also be non-linear
or anisotropic.

  - $f$ is a volumetric source term


### Constitutive Models

Most of the solvers require a constitutive model to be provided and its
parameters populated. This is to allow flexibility in defining / redefining 
solvers during a model calculation.

We need a diffusion model. We can look at the documentation first. 

In [8]:
uw.constitutive_models.DiffusionModel.view()


```python
class DiffusionModel(Constitutive_Model)
...
```
```python
diffusion_model = DiffusionModel(dim)
diffusion_model.material_properties = diffusion_model.Parameters(diffusivity=diffusivity_fn)
scalar_solver.constititutive_model = diffusion_model
```
$$q_{i} = \kappa_{ij} \cdot \frac{\partial \phi}{\partial x_j}$$

where $\kappa$ is a diffusivity, a scalar constant, `sympy` function, `underworld` mesh variable or
any valid combination of those types. Access the constitutive model using:

```python
flux = diffusion_model.flux(gradient_matrix)
```
---


### Problem setup

In [9]:
# The steady-state diffusion
poisson = uw.systems.Poisson(mesh, u_Field=T_soln)
poisson.constitutive_model = uw.constitutive_models.DiffusionModel



In [11]:
# Set the diffusivity
kappa = uw.function.expression(
                    r'\upkappa',
                    sym=nd(1.0 * u.centimetre**2 / u.hour),
                    description="thermal diffusivity"
                        )

poisson.constitutive_model.Parameters.diffusivity = kappa

In [12]:
T_t = uw.function.expression(
                    r'T_t',
                    sym=nd(top_Temp),
                    description="top temperature"
                        )

T_b = uw.function.expression(
                    r'T_b',
                    sym=nd(base_Temp),
                    description="base temperature"
                        )

In [14]:
# Add boundary conditions
poisson.add_essential_bc([T_b], "Bottom")
poisson.add_essential_bc([T_t], "Top"  )

#### Check the system setup by using view()

In [15]:
poisson.view()

**Class**: <class 'underworld3.systems.solvers.SNES_Poisson'>

### Poisson system solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

*Where:*

<IPython.core.display.Latex object>

#### Boundary Conditions

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Bottom | $\left[\begin{matrix}{ T_b \hspace{ 0.04pt } }\end{matrix}\right]  $ | 
| **essential** | Top | $\left[\begin{matrix}{ T_t \hspace{ 0.03pt } }\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

In [16]:
# Solve the system
poisson.petsc_options["snes_converged_reason"] = None

poisson.solve(zero_init_guess=True)

  Nonlinear Solver_2_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


#### Check the Jacobian
It's a little bit hidden, but is available if you feel like checking:

In [17]:
display(poisson._G3)

Matrix([
[87.66,     0],
[    0, 87.66]])

#### View the result

In [18]:
from mpi4py import MPI

if MPI.COMM_WORLD.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, T_soln.sym)

    pl = pv.Plotter(window_size=(1000, 500), shape=(1, 1))


    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1,
        show_scalar_bar=True,
    )

    pl.show()


Widget(value='<iframe src="http://localhost:61510/index.html?ui=P_0x31c24a6e0_1&reconnect=auto" class="pyvista…